# Encrypted Training

## Imports

In [1]:
import time
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *

from concrete.ml.sklearn import SGDClassifier

In [2]:
# Set random seed for reproducibility
np.random.seed(0)

## Load Data

In [3]:
# Load the iris dataset
x, y = load_iris(return_X_y=True)

# Make the problem binary (multiclass classification is not supported yet)
x = x[y != 0]
y = y[y != 0]

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=10)

# Scale the features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## Modelling

In [4]:
# Parameters range
parameters_range = (-1.0, 1.0)

### Non-Encrypted Training

In [5]:
# Initialize the model
model = SGDClassifier(
    n_bits=2,
    max_iter=50,
    fit_encrypted=False,
    parameters_range=parameters_range
)

# Train the model in the clear
train_time = time.time()
model = model.fit(x_train, y_train)
train_time_clear = time.time() - train_time
print(f"Training time: {round(train_time, 2)}")
print()

# Compile the model
model.compile(x_train)

# Evaluate the model
evaluation_time = time.time()
y_pred_clear = model.predict(x_test, fhe='execute')
evaluation_time_clear = time.time() - evaluation_time
print(f"Evaluation time: {round(evaluation_time, 2)}")
print()

# Accuracy
acc_clear = accuracy_score(y_test, y_pred_clear)
print(f"Accuracy: {acc_clear}")

Training time: 1711029024.24

Evaluation time: 1711029024.45

Accuracy: 0.8


### Encrypted Training

In [6]:
# Initialize the model
model = SGDClassifier(
    n_bits=2,
    max_iter=50,
    fit_encrypted=True,
    parameters_range=parameters_range
)

# Train the model in FHE
train_time = time.time()
model = model.fit(x_train, y_train, fhe='execute')
train_time_fhe = time.time() - train_time
print(f"Training time: {round(train_time, 2)}")
print()

# Compile the model
model.compile(x_train)

# Evaluate the model
evaluation_time = time.time()
y_pred_fhe = model.predict(x_test, fhe='execute')
evaluation_time_fhe = time.time() - evaluation_time
print(f"Evaluation time: {round(evaluation_time, 2)}")
print()

# Accuracy
acc_fhe = accuracy_score(y_test, y_pred_fhe)
print(f"Accuracy: {acc_fhe}")

Training time: 1711029024.49

Evaluation time: 1711029295.5

Accuracy: 0.9


## Comparison

In [7]:
# Train time comparison
print(f"Training time (clear): {train_time_clear}")
print(f"Training time (FHE): {train_time_fhe}")
print(f"Slowdown: {train_time_fhe / train_time_clear}")

Training time (clear): 0.09960794448852539
Training time (FHE): 270.89286184310913
Slowdown: 2719.590924540315


In [8]:
# Evaluation time comparison
print(f"Evaluation time (clear): {evaluation_time_clear}")
print(f"Evaluation time (FHE): {evaluation_time_fhe}")

Evaluation time (clear): 0.025860071182250977
Evaluation time (FHE): 0.022551774978637695


In [9]:
# y_pred comparison
print(f"y_pred (clear): {y_pred_clear}")
print(f"y_pred (FHE): {y_pred_fhe}")
print(f"y_pred comparison: {y_pred_clear == y_pred_fhe}")

y_pred (clear): [1 2 1 2 2 2 2 1 2 2]
y_pred (FHE): [1 2 1 2 2 2 1 1 2 2]
y_pred comparison: [ True  True  True  True  True  True False  True  True  True]


In [10]:
# Accuracy comparison
print(f"Accuracy (clear): {acc_clear}")
print(f"Accuracy (FHE): {acc_fhe}")

Accuracy (clear): 0.8
Accuracy (FHE): 0.9
